In [1]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib

#!python -m spacy download pt

import asyncio
from pymongo import MongoClient


DB_HOST_DEV = "mongo"
DB_PASSWORD_DEV = "example"
DB_PORT_DEV = "27017"
DB_DATABASE_DEV = "teste_data"

MONGODB_HOST = "127.0.0.1"  # Ou o endereço IP/nome do servidor MongoDB
MONGODB_PORT = 27017  # Ou a porta correta em que o MongoDB está configurado para ouvir

In [2]:
DB_URL = f"""mongodb://{MONGODB_HOST}:{DB_PASSWORD_DEV}
@{DB_HOST_DEV}:{DB_PORT_DEV}/"""

client = MongoClient("localhost", 27017)
database = client["customer-care-db"]


script_collection = database["script"]

In [3]:
nlp = spacy.load("pt_core_news_sm")

In [4]:
def remove_stopwords(text):
    doc = nlp(text)
    filtered_words = [token.text for token in doc if not token.is_stop]
    return " ".join(filtered_words)

In [5]:
def get_descriptions_dataframe():
    try:
        results = script_collection.find({})
        descriptions = []
        descriptions = [i["problem"] for i in list(results)]
        data = pd.DataFrame(descriptions).reset_index(drop=True)
        data["description"] = data["description"].apply(lambda x: x.lower())
        data["description"] = data["description"].apply(remove_stopwords)
        return data
    except Exception as e:
        raise (f"Error loading or processing data: {e}") from e

In [6]:
data = get_descriptions_dataframe()

In [7]:
# Instantiate the TF-IDF vectorizer
vectorizer = TfidfVectorizer(lowercase=True, strip_accents="unicode")

vectorizer = vectorizer.fit(data["description"])

# Apply TF-IDF on the text dataset
tfidf_matrix = vectorizer.fit_transform(data["description"])

# Get the feature names
feature_names = vectorizer.get_feature_names_out()

# Create a new dataframe with the TF-IDF features
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

In [8]:
# Save model

modelo_file = "modelo_vectorizer.joblib"
joblib.dump(vectorizer, modelo_file)

['modelo_vectorizer.joblib']

In [9]:
# Exemplo de entrada do usuário
user_input = "Estou com alguns problemas com meu seguro."

user_input = remove_stopwords(user_input)

In [10]:
modelo_carregado = joblib.load(modelo_file)

In [11]:
# Vetorização do input do usuário
input_vector = modelo_carregado.transform([user_input])

# Apply TF-IDF on the text dataset
description_matrix = modelo_carregado.transform(data["description"])

# Cálculo da similaridade de cosseno entre o input do usuário e cada descrição
similarity_scores = cosine_similarity(input_vector.toarray(), description_matrix)

In [12]:
most_similar_index = similarity_scores.argmax()

In [13]:
print(
    f"Matriz de similaridade {similarity_scores}\n",
    f"Indice do vetor: {most_similar_index}",
)

Matriz de similaridade [[0.         0.31270025 0.         0.3715104  0.37660176]]
 Indice do vetor: 4


In [14]:
# Resultado
most_similar_description = data["description"][most_similar_index]
print("Descrição mais similar:", most_similar_description)

Descrição mais similar: cancelamento seguro : cliente deseja cancelar seguro contratado anteriormente . precisa orientações processo cancelamento possíveis reembolsos .


In [15]:
data

,id,description
0,101,sinistro automotivo : cliente sofreu acidente ...
1,102,contratação seguro residencial : cliente inter...
2,103,atualização dados cadastrais : cliente precisa...
3,104,esclarecimento cobertura : cliente possui segu...
4,105,cancelamento seguro : cliente deseja cancelar ...
